# tutorial of pytorch by myself

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
import torch
import torch.nn as nn

## Detahch

PyTorch Detach creates a sensor where the storage is shared with another tensor with no grad involved, and thus a new tensor is returned which has no attachments with the current gradients. 
https://www.educba.com/pytorch-detach/

The expected derivative is $\frac{\partial i}{\partial a}=4a^3+6a^5=224$. 

- If we detach the $c$ from the computational graph, all the backward error propagation trough the node of $c$ will disapear. Thus, $\frac{\partial i}{\partial a}=4a^3=32$. 
- If we block all the nodes, the error back propogation will be stopped totally, thus no derivative  $\frac{\partial i}{\partial a}$ can be calculate, there will be an error.

In [2]:
a=torch.tensor([2.0], requires_grad=True)
b=a**4
c=a**6
i=(b+c).sum()
i.backward()
print(a.grad)
print(4*a**3+6*a**5)
print('detach c from the computational graph')
a=torch.tensor([2.0], requires_grad=True)
b=a**4
c=a**6
c=c.detach()
i=(b+c).sum()
i.backward()
print(a.grad)
print(4*a**3)

tensor([224.])
tensor([224.], grad_fn=<AddBackward0>)
detach c from the computational graph
tensor([32.])
tensor([32.], grad_fn=<MulBackward0>)


In [3]:
print('block all the nodes b,c')
a=torch.tensor([2.0], requires_grad=True)
b=a**4
c=a**6
b=b.detach()
c=c.detach()
i=(b+c).sum()
i.backward()
print(a.grad)
print(4*a**3)

block all the nodes b,c


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

Detach method create a tensor which share the same storage with the other. If we modify the detached tensor, this calculation can't be tracked. There will be error: 
- 'one of the variables needed for gradient computation has been modified by an inplace operation'

Torch clone method will fix it.

In [4]:
m = torch.arange(5., requires_grad=True)
n = m**2
o = m.detach()
o.zero_()
print(m)
n.sum().backward()
print(m.grad)

tensor([0., 0., 0., 0., 0.], requires_grad=True)


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [5]] is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [ ]:
m = torch.arange(5., requires_grad=True)
n = m**2
o = m.detach().clone()
o.zero_()
print(m)
n.sum().backward()
print(m.grad)

## Delete or overwriting variables

A non-leaf tensor doesn't need to be saved to perform the derivative, or error backpropagation. Deleting. or overwriting a non leaf tensor is fine  

In [5]:
print('delete b ')
a=torch.tensor([2.0], requires_grad=True)
b=a**3
c=b**2
del b
c.backward()
print(a.grad)
print(6*2**5)

delete b 
tensor([192.])
192


In [6]:
print('overwrite b ')
a=torch.tensor([2.0], requires_grad=True)
b=a**3
b=b**2
b.backward()
print(a.grad)
print(6*2**5)

overwrite b 
tensor([192.])
192


Can't overwrite a leaft variable

In [7]:
a=torch.tensor([2.0], requires_grad=True)
b=a**3
c=b**2
c.backward()
print(a.grad)
print(6*a**5)

print('delete previous a by overwriting by itself')
a=torch.tensor([2.0], requires_grad=True)
a=a**3
print(a)
c=a**2
c.backward()
print(a.grad)
print('now a is treated as one non leaves tensor, there is no grad')


print('delete previous a by overwriting by itself, but with additional operation')
a=torch.tensor([2.0], requires_grad=True)
a=a.clone()**3
# a.requires_grad=True
c=a**2
c.backward()
print(a.grad)
print('now a is treated as one non leaves tensor, there is no grad')





tensor([192.])
tensor([192.], grad_fn=<MulBackward0>)
delete previous a by overwriting by itself
tensor([8.], grad_fn=<PowBackward0>)
None
now a is treated as one non leaves tensor, there is no grad
delete previous a by overwriting by itself, but with additional operation
None
now a is treated as one non leaves tensor, there is no grad


/Users/user/opt/anaconda3/envs/spytorch/lib/python3.10/site-packages/torch/_tensor.py:1013: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_croot-udngs7fm/pytorch_1648016055234/work/build/aten/src/ATen/core/TensorBody.h:417.)
  return self._grad


In [ ]:
print('block all the nodes b,c')
a=torch.tensor([2.0], requires_grad=True)
a=a**3
print(a)
c=a**2
c.backward()
print(a)
print(a.grad)

## How to modify leaf variable, but not tracking by autograd

In [ ]:
a=torch.tensor([2.0], requires_grad=True)
with torch.no_grad():
    a.pow_(3) # where add_ is an inplace operation
b=0.5*a**2
b.backward()
print(a.grad)
print('a**3 is not tracked by the autograd')

In [ ]:
a=torch.tensor([2.0], requires_grad=True)
c=a**3
b=0.5*c**2
b.backward()
print(a.grad)
print(3*a**5)
print('a**3 is tracked by the autograd')

In [ ]:
plot_topic='mem'
for i in range(1, len(net.layer_struct)):
    # the neuron states
    neuron_states=getattr(net, 'hist_{}{}_rec'.format(plot_topic, i))[epoch_index][sampling_index][example_index][time_range_time_step[0]:time_range_time_step[1],:]

    # load the post spk
    output=getattr(net, 'hist_spk{}_rec'.format(i))[epoch_index][sampling_index][:,time_range_time_step[0]:time_range_time_step[1],:]
    output_list=tensor_to_spike_lists(output,net.time_step)

    # load the pre spk
    if i-1==0:
        input=getattr(net, 'spk{}_rec'.format(i-1))[sampling_index][:,time_range_time_step[0]:time_range_time_step[1],:]
        print(input.size())
        input_list=tensor_to_spike_lists(input,net.time_step)
    else:
        input=getattr(net, 'hist_spk{}_rec'.format(i-1))[epoch_index][sampling_index][:,time_range_time_step[0]:time_range_time_step[1],:]
        input_list=tensor_to_spike_lists(input,net.time_step)
    
    fig, axs = plt.subplots(net.layer_struct[i], sharex=True, sharey=True)
    x=np.arange(0, net.duration, net.time_step)[time_range_time_step[0]:time_range_time_step[1]]
    if net.layer_struct[i]>1:
        for neuron_index in range(net.layer_struct[i]):
            # plot the neuron state
            axs[neuron_index].plot(x, neuron_states[:,neuron_index])
            # plot the post spk
            axs[neuron_index].eventplot(output_list[example_index][neuron_index], lineoffsets=0.5, linelengths=1, color='red',linestyles='--')

            # plot the pre spk
            for neuron_index_pre in range(net.layer_struct[i-1]):
                w=getattr(net, 'hist_w{}_rec'.format(i))[epoch_index][sampling_index][neuron_index_pre, neuron_index]
                axs[neuron_index].eventplot(input_list[example_index][neuron_index_pre], lineoffsets=0.5*w, linelengths=w, color='green',linestyles='-.')
    else:
      axs.plot(x, neuron_states[:,0])
      axs.eventplot(output_list[example_index][0], lineoffsets=0.5, linelengths=1, color='red', linestyles='--')
      for neuron_index_pre in range(net.layer_struct[i-1]):
          w=getattr(net, 'hist_w{}_rec'.format(i))[epoch_index][sampling_index][neuron_index_pre,0]
          axs.eventplot(input_list[example_index][neuron_index_pre], lineoffsets=0.5*w, linelengths=w, color='green',linestyles='-.')



    fig.supxlabel('time (ms)')
    fig.supylabel('{}'.format(plot_topic))
    fig.suptitle('{} of layer {}'.format(plot_topic,i))